In [1]:
import os, re
import pandas as pd
import numpy as np
from konlpy.tag import Okt
okt=Okt()

In [2]:
path='/data/yogiyo/pr_data/hufs/'
shop=pd.read_csv(path+'hufs_shop3.csv')

In [3]:
del shop['Unnamed: 0']

In [4]:
def find_in(x):
    if x !=x:
        return False
    find=['리뷰작성','리뷰 작성','리뷰 이벤트','리뷰이벤트','후기이벤트','후기 이벤트','리뷰','서비스','써비스','참여','증정','약속']
    cnt=0
    for tok in find:
        if re.search(tok, x):
            cnt+=1
        if cnt ==2:
            return True
    return False

In [5]:
def review_line(notice,isservice):
    if isservice==False:
        return None
    find=['리뷰','이벤트','요청','사항','서비스','참여','찜','별 점','별점','꼭',
      '제공','선택','약속','작성','결제','닉네임','사진',
     '써비스','랜덤','후기','써비스','증정','안심','번호','포토','꾹']
    
    notice_split=notice.split('\n')
    extract=''
    for sent in notice_split:
        for word in find:
            if re.search(word,sent):
                extract=extract+'\n'+sent
                break
    return extract

In [6]:
shop['isservice']=shop['notice'].map(lambda x: find_in(x))
shop['reviewevent']=shop.apply(lambda x: review_line(x.notice,x.isservice),axis=1)

In [7]:
def extract_photo(sent):
    if sent == None :
        return None
    extract=None
    sent_split=sent.split('\n')
    find=['사진','포토']
    for sent in sent_split:
        for word in find:
            if re.search(word,sent):
                if extract==None:
                    extract=sent
                else:
                    extract=extract+'\n'+sent
                break
    return extract

In [8]:
def extract_star(sent):
    if sent == None :
        return None
    
    
    extract=None
    sent_split=sent.split('\n')
    find=['별']
    for sent in sent_split:
        token=okt.morphs(sent)
        for word in find:
            if word in token:
                if extract==None:
                    extract=sent
                else:
                    extract=extract+'\n'+sent
                break
    return extract

In [9]:
shop['photo']=shop.reviewevent.map(lambda x: extract_photo(x))

In [10]:
shop['star5']=shop.reviewevent.map(lambda x: extract_star(x))

In [18]:
def make_catagory(isservice,photo,star5):
    if not isservice:
        return 4
    cnt=0
    if photo != None:
        cnt+=1
    if star5 != None:
        cnt+=2
    return cnt

In [19]:
shop['catagory']=shop.apply(lambda x: make_catagory(x.isservice,x.photo,x.star5),axis=1)

In [20]:
pd.set_option('display.max_row', None)
shop

shop  star  review_cnt  \
0      엉터리양평해장국&감자탕곱창전골   4.6         107   
1          호세야오리바베큐-이문점   5.0           5   
2             라와마라탕-3호점   4.9         882   
3               명품수제마카롱   4.7          71   
4               신가네동춘찜닭   4.8         133   
5              아빠분식-회기점   5.0           2   
6              형제집육회&연어   5.0           1   
7             PB델리-중랑역점   5.0           1   
8                 망우로30   5.0           2   
9             강정과골뱅이강정골   4.0           1   
10                김환쪽갈비   4.7         318   
11            봉추찜닭-사가정점   4.7         953   
12            성북영양왕족발전문   4.4         199   
13             리김밥-답십리점   4.7          27   
14          알통떡강정-서울장안점   4.2           6   
15           족발야시장-답십리점   4.8         408   
16                 정석김밥   4.6          38   
17          팔자피자&골든에그치킨   4.7          15   
18            신화쭈꾸미-회기점   4.8         243   
19          피자알볼로-이문회기점   4.7        1041   
20                 외대막창   4.9          22   
21         제주딱새우회-동대문구점   4.9          43   
22            마라삼국지-묵동점   4.3           8   
23            떡대떡볶이-중랑점   4.7         229   
24                  분99   4.9          88   
25           빙수목장-월계공릉점   5.0          42   
26         놀부부대찌개-광운대역점   4.6         430   
27        ERIsHouse-중랑점   4.1          59   
28             피자링크-회기점   4.6          47   
29           싸움의고수-경희대점   4.6         627   
30         369두마리치킨-면목점   4.4         153   
31           명동아구찜-묵동본점   4.9          11   
32              푸라닭-월곡점   4.9         109   
33              하루원샤브샤브   4.9          22   
34           이백장돈가스-월계점   5.0          16   
35                  하루미   4.4         160   
36          미스터도시락-장안동점   4.9         167   
37                 야무진닭   4.6          16   
38             우리식당-용두점   5.0           2   
39                낙지랑아구   4.9          96   
40                하마쌀국수   4.8          21   
41          야식King족발&닭발   5.0           2   
42                 휘경곱창   4.4           7   
43                  모참치   5.0          12   
44             고기극찬-중랑점   4.9          52   
45          더꼬치다-동대문전농점   5.0           1   
46                 샐러데이   4.6          55   
47           파스타입니다-길음점   4.9         117   
48                치킨의전설   4.1         250   
49           반이학생마라탕마라반   4.6         406   
50          투썸플레이스-경희대점   4.8          10   
51          우아한혼술식당-월계점   4.6          25   
52         꼬꼬방집으로삼겹-장안점   4.9          15   
53           끼니매운갈비찜전문점   4.9           8   
54            김밥천국-중랑구점   4.4        4593   
55          한솥도시락-휘경주공점   3.9           9   
56                 해원수산   4.6         530   
57                뚠뚠마카롱   5.0           3   
58              CF숯불바베큐   4.9         116   
59       킹콩부대찌개-장안아트몰링점   5.0           7   
60             까로치아-용두점   4.7          22   
61            파스타리아-전농점   4.8         120   
62          멕시카나-장안답십리점   4.8        1530   
63                 회기버거   4.8          44   
64            마초떡볶이-월계점   4.3          32   
65       리얼안심&찹쌀탕수육-석계점   4.7         567   
66        두마리찜닭두찜-성북석관점   4.9        3751   
67                   미성   5.0           3   
68                 신미식성   4.7         456   
69           늘통큰감자탕-전농점   5.0           6   
70      킹부대찌개테이크아웃-석관동점   5.0           1   
71             여기야춘천닭갈비   5.0           6   
72              좋은맛-장안점   5.0           1   
73            신품떡볶이-석관점   4.4         206   
74              태풍참치&스시   4.8         126   
75             도야족발-공릉점   4.8         235   
76           PB델리-신이문역점   5.0           3   
77               또곱창-본점   4.8           4   
78              짜장마을-본점   4.4        2621   
79          리틀하노이-동부시장점   4.7         405   
80               회오징어나라   4.4           8   
81            라멘전문점&라멘야   1.0           1   
82               한첩갈비찜닭   4.7           3   
83        묵은지매운갈비찜-답십리점   4.5          14   
84             랑주317-본점   4.7          13   
85          바푸리포쌀국수 장안점   4.7         125   
86           20년전통외대닭갈비   4.2           5   
87           직화돼지불백-석관점   4.4          19   
88            